<a href="https://colab.research.google.com/github/scenaristeur/igora/blob/main/notebooks/llama_cpp_python_fr.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Téléchargement d'un modèle de langage (dolphin) au format .GGUF depuis HuggingFace / TheBloke https://huggingface.co/TheBloke dans le dossier ./models

In [ ]:
!mkdir models
!wget -O ./models/dolphin-2.2.1-mistral-7b.Q2_K.gguf https://huggingface.co/TheBloke/dolphin-2.2.1-mistral-7B-GGUF/resolve/main/dolphin-2.2.1-mistral-7b.Q2_K.gguf?download=true

# Installation de llama-cpp-python version CPU

In [ ]:
!pip install llama-cpp-python[server] --extra-index-url https://abetlen.github.io/llama-cpp-python/whl/cpu

# On peut maintenant poser une question au modèle de langage en suivant la démo
https://llama-cpp-python.readthedocs.io/en/latest/

In [ ]:
from llama_cpp import Llama
llm = Llama(
      model_path="./models/dolphin-2.2.1-mistral-7b.Q2_K.gguf",
      # n_gpu_layers=-1, # Uncomment to use GPU acceleration
      # seed=1337, # Uncomment to set a specific seed
      # n_ctx=2048, # Uncomment to increase the context window
)
output = llm(
      "Nomme les planètes du système solaire.", # Prompt
      max_tokens=150, # Generate up to 32 tokens, set to None to generate up to the end of the context window
      #stop=["Q:", "\n"], # Stop generating just before the model would generate a new question
      echo=False, # Echo the prompt back in the output
      stream=True
) # Generate a completion, can also call create_completion
print(output)
reponse = output['choices'][0]["text"]
print("\nRéponse", reponse)